In [1]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [4]:
ciemat_template_dict_list = []
this_wvf = []
# ciemat_template_dict = {}
ciemat_templates_model = os.listdir("../template/CIEMAT")
# print(ciemat_templates_model)
for model in ciemat_templates_model:
    # print("Loading model: {}".format(model))
    ciemat_templates = os.listdir("../template/CIEMAT/"+model)
    for ciemat_template in ciemat_templates:
        if "Laser" in ciemat_template:
            wvf_type = "LASER"
        if "SPE" in ciemat_template:
            wvf_type = "SPE"
        
        # print("Loading template: {}".format(ciemat_template))
        if ".npz" in ciemat_template:
            short_wvf = np.load("../template/CIEMAT/"+model+"/"+ciemat_template, allow_pickle=True)["arr_0"][0]
            # print(this_wvf)

        if ".root" in ciemat_template:
            root_file = uproot.open("../template/CIEMAT/"+model+"/"+ciemat_template)
            short_wvf = root_file[root_file.keys()[0]].to_numpy()[0]
            # print(this_wvf)
            
        if len(short_wvf) < len(this_wvf):
            this_wvf = np.concatenate((short_wvf,np.zeros(len(this_wvf)-len(short_wvf))))
            max_length = len(this_wvf)
        else:
            this_wvf = short_wvf
            max_length = len(this_wvf)
            # print(len(this_wvf))
        if np.argmax(this_wvf) != 200:
            this_wvf = np.roll(this_wvf,200-np.argmax(this_wvf))
            
        ciemat_template_dict = {"INST": "CIEMAT",
                                "NAME": ciemat_template,
                                "MODEL": model,
                                "TYPE": wvf_type,
                                "ADC":  this_wvf.tolist(),
                                "TIME": 4e-9*np.arange(len(this_wvf))}
        ciemat_template_dict_list.append(ciemat_template_dict)

# print(ciemat_template_dict)
ciemat_df = pd.DataFrame(ciemat_template_dict_list).explode(["ADC","TIME"])
px.line(ciemat_df[ciemat_df["TYPE"]=="LASER"],x="TIME",y="ADC",color="NAME",facet_col="MODEL",facet_row="TYPE",title="CIEMAT Templates")

In [3]:
milano_template_dict_list = []
milano_templates_model = os.listdir("../template/MILANO")
for model in milano_templates_model:
    milano_templates = os.listdir("../template/MILANO/"+model)
    for milano_template in milano_templates:
        this_wvf = np.loadtxt("../template/MILANO/"+model+"/"+milano_template)
        this_wvf = np.concatenate((np.zeros(100),this_wvf))
        this_wvf = np.roll(this_wvf,50-np.argmax(this_wvf))
        milano_template_dict = {"INST": "MILANO",
                                "NAME": milano_template,
                                "MODEL": model,
                                "TYPE": "SPE",
                                "ADC":  this_wvf.tolist(),
                                "TIME": 16e-9*np.arange(len(this_wvf))}
        milano_template_dict_list.append(milano_template_dict)
milano_df = pd.DataFrame(milano_template_dict_list).explode(["ADC","TIME"])
px.line(milano_df[milano_df["TYPE"]=="SPE"],x="TIME",y="ADC",color="NAME",facet_col="MODEL",facet_row="TYPE",title="CIEMAT Templates")

In [26]:
df = pd.concat([ciemat_df,milano_df])
fig = px.line(df[(df["TYPE"]=="SPE")*(df["MODEL"]=="HPK")],x="TIME",y="ADC",color="NAME",facet_col="MODEL",title="Templates Comparison")
fig.update_xaxes(matches="x", showticklabels=True,mirror=False,showline=True)
fig.update_yaxes(matches=None,showticklabels=True,mirror=False,showline=True)
fig.update_layout(template="presentation",height=500)
fig.show()
# plotly.offline.plot(fig, filename='file.html')

'file.html'

In [37]:
test_df = pd.concat([pd.DataFrame(ciemat_template_dict_list),pd.DataFrame(milano_template_dict_list)])
# test_df["ADC"].astype(float)
# test_df["TIME"].astype(float)

In [47]:
px.line(test_df.explode(["TIME","ADC"]), x="TIME", y="ADC", color="NAME", facet_col="MODEL", facet_row="TYPE", title="Templates Comparison")

TypeError: 'Int64Index' object is not callable